In [3]:
import os
import io
import pickle
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseUpload
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import time

class GoogleDriveUploader:
    def __init__(self, credentials_file='credentials.json', token_file='token.pickle'):
        """
        구글 드라이브 업로더 초기화 (OAuth 2.0 + 토큰 자동 저장)
        
        Args:
            credentials_file (str): OAuth 2.0 클라이언트 자격 증명 JSON 파일
            token_file (str): 액세스 토큰 저장 파일 (자동 생성됨)
        """
        self.credentials_file = credentials_file
        self.token_file = token_file
        self.scopes = ['https://www.googleapis.com/auth/drive.file']
        self.service = self._authenticate()
    
    def _authenticate(self):
        """OAuth 2.0으로 구글 드라이브 API 인증 (토큰 자동 관리)"""
        creds = None
        
        # 1. 기존 토큰 파일 확인 및 로드
        if os.path.exists(self.token_file):
            try:
                with open(self.token_file, 'rb') as token:
                    creds = pickle.load(token)
                print("🔑 기존 인증 토큰을 발견했습니다.")
            except Exception as e:
                print(f"⚠️  토큰 파일 로드 실패: {e}")
                print("🔄 새로운 인증을 진행합니다.")
        
        # 2. 토큰 유효성 검사 및 갱신
        if creds:
            if creds.valid:
                print("✅ 기존 토큰이 유효합니다. 자동 로그인 완료!")
            elif creds.expired and creds.refresh_token:
                try:
                    print("🔄 만료된 토큰을 갱신 중...")
                    creds.refresh(Request())
                    print("✅ 토큰 갱신 완료!")
                    
                    # 갱신된 토큰 저장
                    with open(self.token_file, 'wb') as token:
                        pickle.dump(creds, token)
                        
                except Exception as e:
                    print(f"❌ 토큰 갱신 실패: {e}")
                    print("🔄 재인증이 필요합니다.")
                    creds = None
            else:
                print("❌ 토큰이 만료되었고 갱신할 수 없습니다.")
                creds = None
        
        # 3. 새로운 인증 진행 (필요한 경우에만)
        if not creds or not creds.valid:
            if not os.path.exists(self.credentials_file):
                print(f"❌ 자격 증명 파일을 찾을 수 없습니다: {self.credentials_file}")
                print("📝 Google Cloud Console에서 OAuth 2.0 클라이언트 ID를 생성하고")
                print("   credentials.json 파일을 다운로드하여 프로그램과 같은 폴더에 저장하세요.")
                return None
            
            print("🔐 첫 번째 실행이거나 재인증이 필요합니다.")
            print("🌐 잠시 후 브라우저가 열립니다...")
            
            try:
                flow = InstalledAppFlow.from_client_secrets_file(
                    self.credentials_file, self.scopes)
                
                # 로컬 서버 방식으로 인증 시도
                try:
                    creds = flow.run_local_server(port=0)
                    print("✅ 브라우저 인증 완료!")
                except Exception as e:
                    print(f"로컬 서버 인증 실패: {e}")
                    print("📱 수동 코드 입력 방식으로 전환합니다...")
                    creds = flow.run_console()
                
                # 4. 인증 토큰 저장 (다음번에는 자동 로그인)
                with open(self.token_file, 'wb') as token:
                    pickle.dump(creds, token)
                    print(f"💾 인증 토큰이 '{self.token_file}'에 저장되었습니다.")
                    print("🎉 다음번 실행부터는 자동으로 로그인됩니다!")
                
            except Exception as e:
                print(f"❌ 인증 실패: {e}")
                return None
        
        # 5. API 서비스 생성
        try:
            service = build('drive', 'v3', credentials=creds)
            print("✅ 구글 드라이브 API 연결 완료")
            return service
        except Exception as e:
            print(f"❌ API 서비스 생성 실패: {e}")
            return None
    
    def extract_folder_id_from_url(self, drive_url):
        """구글 드라이브 URL에서 폴더 ID 추출"""
        try:
            if '/folders/' in drive_url:
                folder_id = drive_url.split('/folders/')[1].split('?')[0].split('/')[0]
                return folder_id
            else:
                print("❌ 올바르지 않은 구글 드라이브 폴더 URL입니다.")
                return None
        except Exception as e:
            print(f"❌ URL에서 폴더 ID 추출 실패: {e}")
            return None
    
    def get_folder_info(self, folder_id):
        """폴더 정보 조회"""
        try:
            folder_info = self.service.files().get(
                fileId=folder_id,
                fields='id, name, parents'
            ).execute()
            return folder_info
        except Exception as e:
            print(f"❌ 폴더 정보 조회 실패: {e}")
            return None
    
    def upload_text_file(self, content, filename, folder_id=None):
        """
        텍스트 내용을 파일로 구글 드라이브에 업로드
        
        Args:
            content (str): 업로드할 텍스트 내용
            filename (str): 파일명
            folder_id (str): 업로드할 폴더 ID (기본값: None - 루트 폴더)
            
        Returns:
            str: 업로드된 파일 ID 또는 None
        """
        if not self.service:
            print("❌ 인증되지 않은 상태입니다.")
            return None
            
        try:
            print(f"📤 '{filename}' 업로드 중...")
            
            # 텍스트 내용을 BytesIO 객체로 변환
            file_content = io.BytesIO(content.encode('utf-8'))
            
            # 파일 메타데이터 설정
            file_metadata = {'name': filename}
            if folder_id:
                file_metadata['parents'] = [folder_id]
            
            # 미디어 업로드 객체 생성
            media = MediaIoBaseUpload(
                file_content, 
                mimetype='text/plain',
                resumable=True
            )
            
            # 파일 업로드 실행
            file = self.service.files().create(
                body=file_metadata,
                media_body=media,
                fields='id, name, webViewLink'
            ).execute()
            
            file_id = file.get('id')
            file_name = file.get('name')
            web_link = file.get('webViewLink')
            
            print(f"✅ 파일 업로드 성공!")
            print(f"   📄 파일명: {file_name}")
            print(f"   🆔 파일 ID: {file_id}")
            print(f"   🔗 링크: {web_link}")
            
            return file_id
            
        except Exception as e:
            print(f"❌ 파일 업로드 실패: {e}")
            return None
    
    def upload_local_file(self, file_path, folder_id=None, new_filename=None):
        """로컬 파일을 구글 드라이브에 업로드"""
        if not self.service:
            print("❌ 인증되지 않은 상태입니다.")
            return None
            
        try:
            if not os.path.exists(file_path):
                print(f"❌ 파일을 찾을 수 없습니다: {file_path}")
                return None
            
            filename = new_filename if new_filename else os.path.basename(file_path)
            print(f"📤 '{filename}' 업로드 중...")
            
            file_metadata = {'name': filename}
            if folder_id:
                file_metadata['parents'] = [folder_id]
            
            with open(file_path, 'rb') as file_content:
                media = MediaIoBaseUpload(
                    io.BytesIO(file_content.read()),
                    mimetype='application/octet-stream',
                    resumable=True
                )
            
            file = self.service.files().create(
                body=file_metadata,
                media_body=media,
                fields='id, name, webViewLink'
            ).execute()
            
            file_id = file.get('id')
            file_name = file.get('name')
            web_link = file.get('webViewLink')
            
            print(f"✅ 파일 업로드 성공!")
            print(f"   📄 파일명: {file_name}")
            print(f"   🆔 파일 ID: {file_id}")
            print(f"   🔗 링크: {web_link}")
            
            return file_id
            
        except Exception as e:
            print(f"❌ 파일 업로드 실패: {e}")
            return None
    
    def reset_authentication(self):
        """인증 토큰 삭제 및 재인증 강제"""
        if os.path.exists(self.token_file):
            os.remove(self.token_file)
            print(f"🗑️  기존 토큰 파일 '{self.token_file}' 삭제됨")
        
        print("🔄 다음 실행 시 재인증이 필요합니다.")

def main():
    """메인 함수 - 간편한 사용 예제"""
    
    # OAuth 2.0 클라이언트 자격 증명 JSON 파일 (Google Cloud Console에서 다운로드)
    CREDENTIALS_FILE = 'credentials.json'
    
    # 대상 폴더 URL
    TARGET_FOLDER_URL = 'https://drive.google.com/drive/folders/1ADawmHJ72tr6JgVQIsZ4j7pDS6cjKgi_'
    
    # 업로드할 텍스트 내용
    text_content = f"""
MyTalk 폴더 자동 업로드 테스트

이 파일은 Python 프로그램으로 자동 생성되었습니다.

📋 프로그램 정보:
- 방식: OAuth 2.0 (토큰 자동 저장)
- 첫 실행: 브라우저 인증 1회 필요
- 이후 실행: 완전 자동 (인증 불필요)
- 토큰 유효기간: 약 1시간 (자동 갱신)

📅 생성 시간: {time.strftime('%Y년 %m월 %d일 %H시 %M분 %S초')}

🎯 장점:
✅ 개인 구글 드라이브 사용 가능
✅ 처음 한 번만 인증하면 이후 자동
✅ 토큰 만료 시 자동 갱신
✅ 서비스 어카운트보다 간편

이제 매번 브라우저 인증 없이 자동으로 업로드됩니다!
"""
    
    try:
        print("🚀 Google Drive 자동 업로드 프로그램 시작")
        print("=" * 50)
        
        # 구글 드라이브 업로더 인스턴스 생성
        uploader = GoogleDriveUploader(CREDENTIALS_FILE)
        
        if not uploader.service:
            print("❌ 업로더 초기화 실패")
            print("\n💡 해결방법:")
            print("1. Google Cloud Console에서 OAuth 2.0 클라이언트 ID 생성")
            print("2. credentials.json 파일을 프로그램 폴더에 저장")
            print("3. Google Cloud Console에서 테스트 사용자에 본인 Gmail 추가")
            return
        
        print("=" * 50)
        
        # URL에서 폴더 ID 추출
        folder_id = uploader.extract_folder_id_from_url(TARGET_FOLDER_URL)
        
        if not folder_id:
            print("❌ 폴더 ID 추출 실패")
            return
        
        print(f"📁 대상 폴더 ID: {folder_id}")
        
        # 폴더 정보 확인
        folder_info = uploader.get_folder_info(folder_id)
        if folder_info:
            print(f"📁 대상 폴더명: {folder_info.get('name', '알 수 없음')}")
        
        # 파일명 생성
        filename = f"자동업로드_{int(time.time())}.txt"
        
        # 텍스트 파일 업로드
        file_id = uploader.upload_text_file(
            content=text_content,
            filename=filename,
            folder_id=folder_id
        )
        
        if file_id:
            print("\n" + "=" * 50)
            print("🎉 업로드 완료!")
            print(f"📁 업로드 위치: {TARGET_FOLDER_URL}")
            print("✨ 다음번부터는 브라우저 인증 없이 바로 업로드됩니다!")
        else:
            print("\n❌ 업로드 실패")
            
    except Exception as e:
        print(f"❌ 프로그램 실행 중 오류 발생: {e}")
    
    print("\n" + "=" * 50)
    print("프로그램 종료")

# 토큰 리셋 함수 (필요시 사용)
def reset_auth():
    """저장된 인증 토큰을 삭제하고 재인증 강제"""
    uploader = GoogleDriveUploader()
    uploader.reset_authentication()

if __name__ == "__main__":
    main()
    
    # 재인증이 필요한 경우 아래 주석 해제
    # reset_auth()

🚀 Google Drive 자동 업로드 프로그램 시작
🔑 기존 인증 토큰을 발견했습니다.
✅ 기존 토큰이 유효합니다. 자동 로그인 완료!
✅ 구글 드라이브 API 연결 완료
📁 대상 폴더 ID: 1ADawmHJ72tr6JgVQIsZ4j7pDS6cjKgi_
❌ 폴더 정보 조회 실패: <HttpError 404 when requesting https://www.googleapis.com/drive/v3/files/1ADawmHJ72tr6JgVQIsZ4j7pDS6cjKgi_?fields=id%2C+name%2C+parents&alt=json returned "File not found: 1ADawmHJ72tr6JgVQIsZ4j7pDS6cjKgi_.". Details: "[{'message': 'File not found: 1ADawmHJ72tr6JgVQIsZ4j7pDS6cjKgi_.', 'domain': 'global', 'reason': 'notFound', 'location': 'fileId', 'locationType': 'parameter'}]">
📤 '자동업로드_1755927027.txt' 업로드 중...
✅ 파일 업로드 성공!
   📄 파일명: 자동업로드_1755927027.txt
   🆔 파일 ID: 1ovn426dLPfjq3-akrbhFNRCXuZucJBZ5
   🔗 링크: https://drive.google.com/file/d/1ovn426dLPfjq3-akrbhFNRCXuZucJBZ5/view?usp=drivesdk

🎉 업로드 완료!
📁 업로드 위치: https://drive.google.com/drive/folders/1ADawmHJ72tr6JgVQIsZ4j7pDS6cjKgi_
✨ 다음번부터는 브라우저 인증 없이 바로 업로드됩니다!

프로그램 종료
